In [2]:
!pip install lxml


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Bilal Ahmad\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [3]:
import urllib.request
import requests
import ssl

# Create an SSL context that ignores certificate verification (for development only)
context = ssl._create_unverified_context()

# arXiv API URL for machine learning papers
url = 'https://export.arxiv.org/api/query?search_query=all:machine+learning&start=0&max_results=1'

# Send the request and read the response
with urllib.request.urlopen(url, context=context) as response:
    data = response.read().decode('utf-8')
    print(data)



<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3Amachine%20learning%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:machine learning&amp;id_list=&amp;start=0&amp;max_results=1</title>
  <id>http://arxiv.org/api/REmrP0ijDn1S2ro+OMK7IoW1GgY</id>
  <updated>2025-07-18T00:00:00-04:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">420116</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage>
  <entry>
    <id>http://arxiv.org/abs/1909.03550v1</id>
    <updated>2019-09-08T21:49:42Z</updated>
    <published>2019-09-08T21:49:42Z</published>
    <title>Lecture Notes: Optimization for 

In [4]:
import requests
url= 'http://arxiv.org/pdf/1909.03550v1'
r= requests.get(url= url)

In [13]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langchain_core.tools import tool
from typing import TypedDict
from pydantic import Field
from pypdf import PdfReader
from io import BytesIO
import urllib.request
import ssl
from bs4 import BeautifulSoup
from fpdf import FPDF
from dotenv import load_dotenv

# load the api key
load_dotenv()

# Define state structure
class SummarizerState(TypedDict):
    user_query: str
    keyword: str
    content: list[str]
    summary: str
    report: object

# LangChain model
llm = ChatOpenAI()  # Replace with your OpenAI API key

# Keyword Extraction Node
def keyword_extraction(state: SummarizerState):
    prompt = PromptTemplate(
        template="""
        Extract only 1 keyword (max 2 words) from the query below:
        {user_query}

        Conditions:
        - Combine multi-word keywords with a + sign like machine+learning
        - Do not use more than 2 words
        - Do not return more than 1 keyword

        Example: For "How can AI help in medical diagnosis?", return "medical+AI"
        """,
        input_variables=["user_query"]
    )
    result = llm.invoke(prompt.format(user_query=state['user_query']))
    return {"keyword": result.content.strip()}

# Retrieve PDF Text using arXiv API
def retrieve_docs(state: SummarizerState):
    try:
        query = state["keyword"].replace(" ", "+")
        url = f'https://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results=10'
        context = ssl._create_unverified_context()
        
        with urllib.request.urlopen(url, context=context) as response:
            data = response.read().decode('utf-8')
        
        soup = BeautifulSoup(data, 'xml')
        pdf_url = soup.find('link', attrs={'title': 'pdf'}).get('href')
        
        with urllib.request.urlopen(pdf_url, context=context) as pdf_response:
            pdf_data = pdf_response.read()
        
        doc = PdfReader(BytesIO(pdf_data))
        content = []
        for page in doc.pages:
            text = page.extract_text()
            if text:
                content.append(text)
        
        return {"content": content}
    
    except Exception as e:
        print("Error retrieving document:", e)
        return {"content": ["No document found."]}

# Summarization Node
def summarizer_agent(state: SummarizerState):
    combined_text = "\n".join(state["content"])[:3000]  # Truncate if too long
    prompt = PromptTemplate(
        template="""
        Given the following content, summarize it to answer the user's query: {user_query}

        Then explain a real-world example of how the summarized knowledge can solve a practical problem.

        Content:
        {content}
        """,
        input_variables=["user_query", "content"]
    )
    result = llm.invoke(prompt.format(user_query=state['user_query'], content=combined_text))
    return {"summary": result.content.strip()}

# PDF Report Generation Tool
@tool
def generate_pdf_tool(summary: str) -> str:
    """Generate a well-formatted PDF from the summary and return the file path."""
    pdf = FPDF()
    pdf.add_page()
    
    # Title
    pdf.set_font("Times", "B", 16)
    pdf.cell(0, 10, "Research Summary Report", ln=True, align="C")
    pdf.ln(10)

    # Summary Section
    pdf.set_font("Times", "B", 14)
    pdf.cell(0, 10, "Summary", ln=True)
    pdf.set_font("Times", "", 12)
    pdf.multi_cell(0, 10, summary)

    # Real-World Example Section
    pdf.ln(8)
    pdf.set_font("Times", "B", 14)
    pdf.cell(0, 10, "Real-World Application", ln=True)
    pdf.set_font("Times", "", 12)
    pdf.multi_cell(0, 10,
        "Example: A hospital uses Vision Transformers to detect pneumonia from X-ray images. "
        "Unlike older CNNs, Vision Transformers process the whole image contextually, identifying patterns even in blurry or complex regions. "
        "This reduces missed cases and improves patient outcomes."
    )

    output_path = "Summarized_Report.pdf"
    pdf.output(output_path)
    return output_path

# Final Node to Generate PDF
def generate_report(state: SummarizerState):
    report_path = generate_pdf_tool.invoke({"summary": state["summary"]})
    return {"report": report_path}

# Create the graph
graph = StateGraph(state_schema=SummarizerState)
graph.add_node("keyword_extraction", keyword_extraction)
graph.add_node("retrieve_docs", retrieve_docs)
graph.add_node("summarizer_agent", summarizer_agent)
graph.add_node("generate_report", generate_report)

graph.add_edge(START, "keyword_extraction")
graph.add_edge("keyword_extraction", "retrieve_docs")
graph.add_edge("retrieve_docs", "summarizer_agent")
graph.add_edge("summarizer_agent", "generate_report")
graph.add_edge("generate_report", END)

# Compile the graph
app = graph.compile()

# Run the flow
final_output = app.invoke({
    "user_query": "How is Vision Transformers better than previous Vision models?"
})

print("Final Summary:\n", final_output['summary'])
print("PDF saved to:", final_output['report'])


Final Summary:
 Vision Transformers, such as LW-Transformer, are better than previous vision models due to their ability to reduce the number of parameters and computations while maintaining efficient attention modeling and feature transformation. Experimental results show that LW-Transformer achieves competitive performance for vision-and-language tasks. 
In a real-world example, this improved efficiency of LW-Transformer could be applied in image classification tasks, such as identifying objects in a large dataset of images. By reducing the computational cost while maintaining performance, LW-Transformer can help process and analyze images faster and more accurately in various applications like medical imaging or autonomous driving systems.
PDF saved to: Summarized_Report.pdf
